## Setup and Configuration

In [ ]:
import requests
import pandas as pd
import numpy as np
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuration
API_BASE = 'http://localhost:3001/api'

print("✓ Libraries loaded successfully")
print(f"Timestamp: {datetime.now().isoformat()}")

## Section 1: Basic Protocol Simulation

Let's run a simple simulation of a CRISPR plasmid prep protocol

In [ ]:
# Run a baseline simulation
simulation_params = {
    'protocol': 'plasmid-prep',
    'numRuns': 5,
    'params': {
        'initialYield': 100,
        'yieldLossPerStep': 0.05
    },
    'metrics': ['cost', 'duration', 'yield']
}

response = requests.post(
    f'{API_BASE}/v1/agents/simulate',
    json=simulation_params
)

sim_result = response.json()['simulation']

print(f"✓ Simulation completed: {sim_result['simulationId']}")
print(f"Protocol: {sim_result['protocol']}")
print(f"Number of runs: {sim_result['numRuns']}")
print(f"\nAggregated metrics:")
for metric, stats in sim_result['aggregated'].items():
    print(f"\n{metric.upper()}:")
    for key, val in stats.items():
        print(f"  {key}: {val:.2f}")

## Section 2: Analyze Simulation Results

Convert results to DataFrame for analysis

In [ ]:
# Create DataFrame from simulation runs
runs_data = []
for run in sim_result['runs']:
    runs_data.append({
        'Run': run['runId'],
        'Cost ($)': run['cost'],
        'Duration (min)': run['duration'],
        'Yield (%)': run['yield'] * 100,
        'Success Rate': run['successRate'] * 100
    })

df_runs = pd.DataFrame(runs_data)
print(df_runs.to_string(index=False))
print(f"\nSummary Statistics:")
print(df_runs[['Cost ($)', 'Duration (min)', 'Yield (%)']].describe())

## Section 3: Parameter Variation Study

Test different yield loss parameters to find optimal settings

In [ ]:
# Test different yield loss parameters
yield_loss_params = [0.02, 0.05, 0.10, 0.15, 0.20]
results_by_param = []

for loss_param in yield_loss_params:
    sim_params = {
        'protocol': 'plasmid-prep',
        'numRuns': 3,
        'params': {'yieldLossPerStep': loss_param},
        'metrics': ['yield']
    }
    
    response = requests.post(
        f'{API_BASE}/v1/agents/simulate',
        json=sim_params
    )
    
    result = response.json()['simulation']
    avg_yield = result['aggregated']['yield']['mean'] * 100
    
    results_by_param.append({
        'Yield Loss %': loss_param * 100,
        'Average Final Yield %': avg_yield
    })
    
    print(f"Yield loss {loss_param*100:.1f}%: Average final yield = {avg_yield:.1f}%")

df_params = pd.DataFrame(results_by_param)
print("\n" + df_params.to_string(index=False))

## Section 4: Bottleneck Analysis

Identify steps that are limiting protocol performance

In [ ]:
# Analyze bottlenecks from original simulation
bottlenecks = []
for run in sim_result['runs']:
    if run['bottlenecks']:
        for bottleneck in run['bottlenecks']:
            bottlenecks.append(bottleneck)

if bottlenecks:
    print(f"Identified {len(bottlenecks)} bottleneck(s):")
    from collections import Counter
    bottleneck_counts = Counter(bottlenecks)
    for step, count in bottleneck_counts.most_common():
        print(f"  - {step}: {count} occurrences ({count/len(sim_result['runs'])*100:.1f}% of runs)")
else:
    print("✓ No significant bottlenecks detected")

# Cost analysis
avg_cost = sim_result['aggregated']['cost']['mean']
std_cost = sim_result['aggregated']['cost']['stdDev']
print(f"\nCost Analysis:")
print(f"  Average cost per run: ${avg_cost:.2f}")
print(f"  Cost variability (std dev): ${std_cost:.2f}")
print(f"  Cost range: ${sim_result['aggregated']['cost']['min']:.2f} - ${sim_result['aggregated']['cost']['max']:.2f}")

## Section 5: Recommendations

Based on simulation results, provide actionable insights

In [ ]:
# Generate recommendations
recommendations = []

avg_yield = sim_result['aggregated']['yield']['mean']
if avg_yield < 0.75:
    recommendations.append("⚠️  Low yield (<75%): Consider optimizing reagent concentrations or incubation times")
elif avg_yield > 0.90:
    recommendations.append("✓ Excellent yield (>90%): Current protocol parameters are well-optimized")

yield_variance = sim_result['aggregated']['yield']['stdDev']
if yield_variance > 0.15:
    recommendations.append("⚠️  High yield variability: Tighten control of critical parameters (temperature, timing)")
else:
    recommendations.append("✓ Consistent yield: Protocol is reproducible")

avg_duration = sim_result['aggregated']['duration']['mean']
recommendations.append(f"📊 Average protocol duration: {avg_duration:.0f} minutes")

print("RECOMMENDATIONS:")
for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec}")

## Next Steps

1. **Iterate on parameters**: Use the parameter variation study to identify optimal settings
2. **Run validation simulations**: Test recommended parameters in wet-lab validation
3. **Compare protocols**: Test alternative protocols using similar approach
4. **Document findings**: Record results and lessons learned for future protocols

See notebook `02-agent-demo.ipynb` for AI-driven optimization examples.